In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
#nltk.download('punkt')
#nltk.download('stopwords')

In [3]:
file_path = "chat.html"
with open(file_path, "r", encoding="utf-8") as file:
    html = file.read()

soup = BeautifulSoup(html, "html.parser").decode("utf-8")
conversations = soup.split('{"title": ')

In [4]:
text_list = []
for i in range(1,len(conversations)):
    messages = conversations[i].split('"parts": ["')
    for j in range(2,len(messages)):
        message = messages[j].partition('"]}, "status"')[0]
        text_list.append(message)

In [5]:
text = "\n".join(text_list)
words = word_tokenize(text)
words = [word.lower() for word in words if word.isalnum()]
stop_words = set(stopwords.words("english"))
words = [word for word in words if word not in stop_words]

In [6]:
data = pd.DataFrame({
    'word': list(words)
}).sort_values(by='word').reset_index(drop=True)
data = data.loc[data.word != '0'].reset_index(drop=True)

In [7]:
nouns_path = 'dictionary/nouns/91K nouns.txt'
with open(nouns_path, "r", encoding="utf-8") as file:
    nouns_str = file.read()
nouns = nouns_str.split('\n')

verbs_path = 'dictionary/verbs/31K verbs.txt'
with open(verbs_path, "r", encoding="utf-8") as file:
    verbs_str = file.read()
verbs = verbs_str.split('\n')

adverbs_path = 'dictionary/adverbs/6K adverbs.txt'
with open(adverbs_path, "r", encoding="utf-8") as file:
    adverbs_str = file.read()
adverbs = adverbs_str.split('\n')

adjectives_path = 'dictionary/adjectives/28K adjectives.txt'
with open(adjectives_path, "r", encoding="utf-8") as file:
    adjectives_str = file.read()
adjectives = adjectives_str.split('\n')

dictionary = nouns + verbs + adverbs + adjectives

In [8]:
data = data[data['word'].isin(dictionary)].reset_index(drop=True)
data['noun'] = data['word'].isin(nouns).astype(int)
data['verb'] = data['word'].isin(verbs).astype(int)
data['adverb'] = data['word'].isin(adverbs).astype(int)
data['adjective'] = data['word'].isin(adjectives).astype(int)

In [9]:
data.to_csv('data.csv', index=False)